# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---


# <h1><center>Clase 05: Acceso a datos en internet con `requests`</center></h1>

**Objetivo:** mostrar, con ejemplos prácticos, cómo usar la librería `requests` para obtener datos desde la web (JSON, CSV, binarios), manejar parámetros, errores y guardar los resultados para su análisis con `pandas`.


## 1. Setup

Instalaremos/Importaremos las librerías necesarias. En equipos con Anaconda, ya deberían estar disponibles.


In [5]:
import requests
import pandas as pd
from pathlib import Path
from time import sleep
print("Librerías listas.")

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)


## 2. Descarga de **CSV** desde una URL directa (raw.githubusercontent.com)

Usaremos un dataset público del repositorio **FiveThirtyEight** (college majors).**URL:** `https://raw.githubusercontent.com/fivethirtyeight/data/master/college-majors/recent-grads.csv`

Demostraremos:
- Descarga con `requests.get`
- Guardado local en `data/`
- Carga con `pandas.read_csv`
- Verificación del tamaño descargado


In [3]:
# URL del CSV con los datos
csv_url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/college-majors/recent-grads.csv"
# lugar donde guardarlo en el computador local
out_dir = Path("data")
out_dir.mkdir(exist_ok=True)
csv_path = out_dir / "recent-grads.csv"

# solicitar los datos
r = requests.get(csv_url, timeout=30)
r.raise_for_status()  # Lanza error si el status no es 200
csv_path.write_bytes(r.content)

print("Guardado en:", csv_path.resolve())
print("Tamaño (bytes):", csv_path.stat().st_size)

NameError: name 'Path' is not defined

In [ ]:
df_csv = pd.read_csv(csv_path)
df_csv.head()


## 3. Manejo de **errores** y **timeouts**

Buenas prácticas:
- Usar `timeout=` para evitar que el programa quede colgado.
- Verificar `response.status_code` o usar `raise_for_status()`.
- Capturar excepciones con `try/except` y registrar mensajes útiles.


In [ ]:
bad_url = "https://api.github.com/this/endpoint/does/not/exist"

try:
    r = requests.get(bad_url, timeout=5)
    r.raise_for_status()
except requests.exceptions.HTTPError as e:
    print("HTTPError:", e)
except requests.exceptions.Timeout:
    print("Timeout alcanzado.")
except requests.exceptions.RequestException as e:
    print("Error de red:", e)


## 4. Sesiones y cabeceras personalizadas (`Session`), backoff simple

A veces conviene reutilizar conexiones con `requests.Session()` y establecer **cabeceras** (headers) comunes. También podemos implementar un **reintento simple** (backoff) frente a códigos temporales (p. ej., 429 o 5xx).


In [ ]:
session = requests.Session()
session.headers.update({
    "Accept": "application/vnd.github+json",
    "User-Agent": "imt2200-class-notebook"
})

def get_with_retry(url, params=None, max_retries=3, backoff=2):
    for i in range(1, max_retries+1):
        resp = session.get(url, params=params, timeout=15)
        if resp.status_code == 200:
            return resp
        print(f"Intento {i}: status {resp.status_code}. Reintentando en {backoff} s...")
        sleep(backoff)
    resp.raise_for_status()

# Prueba reutilizando la API de GitHub (misma consulta de antes)
resp_ok = get_with_retry("https://api.github.com/search/repositories", params={
    "q": "geopandas language:python",
    "sort": "stars",
    "order": "desc",
    "per_page": 5
})
pd.json_normalize(resp_ok.json().get("items", []))[["full_name","stargazers_count","html_url"]].head()


## 5. De **JSON anidado** a **tabla** (normalización)

Cuando la respuesta incluye estructuras anidadas, `pandas.json_normalize` ayuda a "aplanar" el JSON.


In [ ]:
headers = {
    "Accept": "application/vnd.github+json",
    "User-Agent": "imt2200-class-notebook"
}
resp = requests.get("https://api.github.com/repos/pandas-dev/pandas/issues",
                    params={"state":"open", "per_page":10},
                    headers=headers, timeout=15)

In [ ]:
issues = resp.json()
issues_df = pd.json_normalize(issues, sep="_", max_level=2)[["number","title","state","user_login","comments","created_at","updated_at","html_url"]]
issues_df.head()


## 6. Guardar resultados procesados

Exportamos a CSV/Parquet para reutilizar los datos luego, sin necesidad de volver a llamar a la API.


In [ ]:
# lugar donde almacenar los datos
proc_dir = Path("outputs")
proc_dir.mkdir(exist_ok=True)

# grabar
issues_df.to_csv(proc_dir / "pandas_issues_sample.csv", index=False)
print("Archivos exportados en:", proc_dir.resolve())


---

### 📌 Buenas prácticas y ética de uso

- **Respeta** los términos de uso de las APIs y sitios web (TOS).  No hagas scraping de sitios que lo prohíben o requieran autenticación.  
- Revisa **límites de rate limit** (p. ej., GitHub limita requests por minuto sin token).  
- **Cachea** o guarda resultados para evitar llamadas innecesarias.  
- **Identifícate** con un `User-Agent` claro y usa `timeout` y reintentos responsables.  
- Cita las **fuentes** y documenta el **proceso** en tu repositorio.

---
